In [60]:
from enum import unique, Enum, auto
from typing import List, Callable, Dict, Any, Tuple

In [61]:
def cost_matrix(first_str: str, second_str: str, key_func: Callable) -> List[List[int]]:
    """
    Generate a cost matrix for `first_str` and `second_str`.
    """
    matrix_rows = list()
    first_len, second_len = len(first_str), len(second_str
                                                )
    previous_row = list(range(second_len + 1))

    for i in range(first_len):
        matrix_rows.append(previous_row)
        current_row = [i + 1]
        for j in range(second_len):
            insertion_count = previous_row[j + 1] + 1
            deletion_count = current_row[j] + 1
            is_equal_key = key_func(first_str[i]) == key_func(second_str[j])
            substitution_count = int(not is_equal_key) + previous_row[j]
            current_row.append(min(insertion_count, deletion_count, substitution_count))
        previous_row = current_row
    else:
        matrix_rows.append(previous_row)

    return matrix_rows

In [62]:
@unique
class Type(str, Enum):
    """
    Operation type for edits.
    """
    MATCH = auto()
    SUBSTITUTION = auto()
    DELETION = auto()
    INSERTION = auto()

def backtrace(first_str: str, second_str: str, matrix_rows: List[List[int]]) -> List[Dict[str, Any]]:
    """
    Back trace and generate edits.
    """
    first_len, second_len = len(first_str), len(second_str)
    i, j = first_len, second_len
    distance_edits = list()
 
    while any((i, j)):
        previous_cost, neighbors = matrix_rows[i][j], list()
        neighbors.append(matrix_rows[i - 1][j - 1])
        if i:
            neighbors.append(matrix_rows[i - 1][j])
        if j:
            neighbors.append(matrix_rows[i][j - 1])

        min_cost = min(neighbors)
        _type = None
        if min_cost == previous_cost:
            i, j, _type= i - 1, j - 1, Type.MATCH
        elif i and j and min_cost == matrix_rows[i - 1][j - 1]:
            i, j, _type = i - 1, j - 1, Type.SUBSTITUTION
        elif i and min_cost == matrix_rows[i - 1][j]:
            i, j, _type = i - 1, j, Type.DELETION
        elif j and min_cost == matrix_rows[i][j - 1]:
            i, j, _type = i, j - 1, Type.INSERTION
        if _type:
            distance_edits.append({'type': str(_type), 'i':i, 'j':j})
    else:
        distance_edits.reverse()

    return distance_edits

In [63]:
def levenshtein(first_str: str, second_str: str) -> Tuple[int, List[Dict[str, Any]]]:
    """
    Calculate a levenshtein distance and edits for `first_str` and `second_str`.
    """
    matrix_rows = cost_matrix(first_str, second_str, hash)
    return matrix_rows[-1][-1], backtrace(first_str, second_str, matrix_rows)

In [64]:
levenshtein('HONDA', 'HYUNDAI')

(3,
 [{'type': 'Type.MATCH', 'i': 0, 'j': 0},
  {'type': 'Type.INSERTION', 'i': 1, 'j': 1},
  {'type': 'Type.SUBSTITUTION', 'i': 1, 'j': 2},
  {'type': 'Type.MATCH', 'i': 2, 'j': 3},
  {'type': 'Type.MATCH', 'i': 3, 'j': 4},
  {'type': 'Type.MATCH', 'i': 4, 'j': 5},
  {'type': 'Type.INSERTION', 'i': 5, 'j': 6}])

In [65]:
levenshtein('kitten', 'kitchen')

(2,
 [{'type': 'Type.MATCH', 'i': 0, 'j': 0},
  {'type': 'Type.MATCH', 'i': 1, 'j': 1},
  {'type': 'Type.MATCH', 'i': 2, 'j': 2},
  {'type': 'Type.INSERTION', 'i': 3, 'j': 3},
  {'type': 'Type.SUBSTITUTION', 'i': 3, 'j': 4},
  {'type': 'Type.MATCH', 'i': 4, 'j': 5},
  {'type': 'Type.MATCH', 'i': 5, 'j': 6}])

In [66]:
levenshtein('kitten', 'bitten')

(1,
 [{'type': 'Type.SUBSTITUTION', 'i': 0, 'j': 0},
  {'type': 'Type.MATCH', 'i': 1, 'j': 1},
  {'type': 'Type.MATCH', 'i': 2, 'j': 2},
  {'type': 'Type.MATCH', 'i': 3, 'j': 3},
  {'type': 'Type.MATCH', 'i': 4, 'j': 4},
  {'type': 'Type.MATCH', 'i': 5, 'j': 5}])